In [1]:
import nltk
import re
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Input

In [2]:
df = pd.read_csv('Dataset/ner_dataset.csv', encoding= 'unicode_escape')
df

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
...,...,...,...,...
1048570,NaN,they,PRP,O
1048571,NaN,responded,VBD,O
1048572,NaN,to,TO,O
1048573,NaN,the,DT,O


In [3]:
ready_df = pd.read_csv('Dataset/ner.csv')
ready_df

,Sentence #,Sentence,POS,Tag
0,Sentence: 1,Thousands of demonstrators have marched throug...,"['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '..."
1,Sentence: 2,Families of soldiers killed in the conflict jo...,"['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,Sentence: 3,They marched from the Houses of Parliament to ...,"['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,Sentence: 4,"Police put the number of marchers at 10,000 wh...","['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,Sentence: 5,The protest comes on the eve of the annual con...,"['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'D...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
...,...,...,...,...
47954,Sentence: 47955,Indian border security forces are accusing the...,"['JJ', 'NN', 'NN', 'NNS', 'VBP', 'VBG', 'PRP$'...","['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'B-gpe..."
47955,Sentence: 47956,Indian officials said no one was injured in Sa...,"['JJ', 'NNS', 'VBD', 'DT', 'NN', 'VBD', 'VBN',...","['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '..."
47956,Sentence: 47957,Two more landed in fields belonging to a nearb...,"['CD', 'JJR', 'VBD', 'IN', 'NNS', 'VBG', 'TO',...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
47957,Sentence: 47958,They say not all of the rockets exploded upon ...,"['PRP', 'VBP', 'RB', 'DT', 'IN', 'DT', 'NNS', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 4 columns):
 #   Column      Non-Null Count    Dtype 
---  ------      --------------    ----- 
 0   Sentence #  47959 non-null    object
 1   Word        1048575 non-null  object
 2   POS         1048575 non-null  object
 3   Tag         1048575 non-null  object
dtypes: object(4)
memory usage: 32.0+ MB


In [5]:
ready_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47959 entries, 0 to 47958
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Sentence #  47959 non-null  object
 1   Sentence    47959 non-null  object
 2   POS         47959 non-null  object
 3   Tag         47959 non-null  object
dtypes: object(4)
memory usage: 1.5+ MB


In [6]:
unique_words = len(np.unique(df['Word']))
unique_tags = df['Tag'].unique()
total_unique_tags = len(np.unique(df['Tag']))
print("total unique words :", unique_words)
print("total unique tags :", total_unique_tags)
print("tags : ", unique_tags)

total unique words : 35178
total unique tags : 17
tags :  ['O' 'B-geo' 'B-gpe' 'B-per' 'I-geo' 'B-org' 'I-org' 'B-tim' 'B-art'
 'I-art' 'I-per' 'I-gpe' 'I-tim' 'B-nat' 'B-eve' 'I-eve' 'I-nat']


In [7]:
X = list(ready_df['Sentence'])
y = list(ready_df['Tag'])
X[:10], y[:10]

(['Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .',
  'Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as " Bush Number One Terrorist " and " Stop the Bombings . "',
  'They marched from the Houses of Parliament to a rally in Hyde Park .',
  'Police put the number of marchers at 10,000 while organizers claimed it was 1,00,000 .',
  "The protest comes on the eve of the annual conference of Britain 's ruling Labor Party in the southern English seaside resort of Brighton .",
  "The party is divided over Britain 's participation in the Iraq conflict and the continued deployment of 8,500 British troops in that country .",
  'The London march came ahead of anti-war protests today in other cities , including Rome , Paris , and Madrid .',
  "The International Atomic Energy Agency is to hold second day of talks in Vienna Wednesday on how to r

In [8]:
print("total sentences: ", len(X))

total sentences:  47959


In [9]:
# to get the max length sentence
maxlen = max(len(s) for s in X)
print(maxlen)

541


In [10]:
# maxlen is too big so we will be taking a small length
maxlen = 110

total_words = unique_words

tokenizer = Tokenizer(num_words=total_words)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)


pickle.dump(tokenizer, open('tokenizer.pkl', 'wb'))

word_index = tokenizer.word_index
vocab_size = len(word_index)
print(f"Vocabulary contains {vocab_size} words\n")

Vocabulary contains 27953 words



In [11]:
X_padded = pad_sequences(sequences,maxlen = maxlen, padding = 'post', truncating = 'post')

In [12]:
X_padded.shape

(47959, 110)

In [13]:
X_padded

array([[ 260,    3,  997, ...,    0,    0,    0],
       [1191,    3,  148, ...,    0,    0,    0],
       [  45, 1838,   21, ...,    0,    0,    0],
       ...,
       [  36,   46, 2760, ...,    0,    0,    0],
       [  45,   26,   41, ...,    0,    0,    0],
       [ 364,   75,   16, ...,    0,    0,    0]], dtype=int32)

In [14]:
tag_to_num = {tag: num for num, tag in enumerate(unique_tags)}

example_tag = 'B-org'
example_tag_num = tag_to_num[example_tag]
print(f"Tag: {example_tag}, Numerical Value: {example_tag_num}")

Tag: B-org, Numerical Value: 5


In [15]:
tag_to_num

{'O': 0,
 'B-geo': 1,
 'B-gpe': 2,
 'B-per': 3,
 'I-geo': 4,
 'B-org': 5,
 'I-org': 6,
 'B-tim': 7,
 'B-art': 8,
 'I-art': 9,
 'I-per': 10,
 'I-gpe': 11,
 'I-tim': 12,
 'B-nat': 13,
 'B-eve': 14,
 'I-eve': 15,
 'I-nat': 16}

In [16]:
tag_to_num['O']

0

In [17]:
def gettags(s):
    return s[2:-2].split("', '")

In [18]:
def tags_list2idx(t):
    t = [tag_to_num[tag] for tag in gettags(t)]
    return t

In [19]:
Y = [tags_list2idx(t) for t in y]

In [20]:
y[1]

"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-per', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']"

In [21]:
Y[1]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [22]:
Y = pad_sequences(maxlen=maxlen, sequences=Y, padding="post", truncating = 'post', value=tag_to_num["O"])
Y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [2, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [23]:
len(X), len(Y)

(47959, 47959)

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(X_padded, Y, test_size=0.2, random_state=1)

X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=1)

In [25]:
print("Total training examples : ", len(X_train))
print("Total validation examples : ", len(X_val))
print("Total test examples : ", len(X_test))

Total training examples :  28775
Total validation examples :  9592
Total test examples :  9592


In [26]:
X_train[100]

array([  25,  199,  662, 2665,    4, 4226,   59,  956,  913, 2480,  168,
         81,    7, 1866,   31,  122,   16,   10,   14,   20,  698,   95,
         12,  662,   32, 1162,  846, 1020,   46,    3,   59, 2263,  193,
          3,  913, 1794,  924,    2,  563,    6,  372,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
      dtype=int32)

In [27]:
Y_train[100]

array([ 0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  7,  0,  0,  0,  3, 10,
        0,  0,  1,  0,  7,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  7, 12, 12,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0], dtype=int32)

In [28]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test))

In [29]:
BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 128

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [30]:
maxlen = 110
maxwords = vocab_size
num_tags = total_unique_tags

model = Sequential([
    Embedding(input_dim=vocab_size+1, output_dim=maxlen, input_length=maxlen),
    Dropout(0.1),
    Bidirectional(LSTM(units = 100, activation = 'tanh', return_sequences = True)),
    Bidirectional(LSTM(units = 100, activation = 'tanh', return_sequences = True, recurrent_dropout=0.1)),
    TimeDistributed(Dense(num_tags, activation = 'softmax'))
])

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 110, 110)          3074940   
                                                                 
 dropout (Dropout)           (None, 110, 110)          0         
                                                                 
 bidirectional (Bidirectiona  (None, 110, 200)         168800    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 110, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed (TimeDistr  (None, 110, 17)          3417      
 ibuted)                                                         
                                                        

In [32]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001, decay=1e-5),
              metrics=['accuracy'])

In [33]:
history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=5)

Epoch 1/5
225/225 [==============================] - 527s 2s/step - loss: 0.2104 - accuracy: 0.9654 - val_loss: 0.1191 - val_accuracy: 0.9697
Epoch 2/5
225/225 [==============================] - 505s 2s/step - loss: 0.1073 - accuracy: 0.9712 - val_loss: 0.1030 - val_accuracy: 0.9717
Epoch 3/5
225/225 [==============================] - 499s 2s/step - loss: 0.0854 - accuracy: 0.9748 - val_loss: 0.0791 - val_accuracy: 0.9760
Epoch 4/5
225/225 [==============================] - 481s 2s/step - loss: 0.0683 - accuracy: 0.9780 - val_loss: 0.0723 - val_accuracy: 0.9776
Epoch 5/5
225/225 [==============================] - 487s 2s/step - loss: 0.0592 - accuracy: 0.9801 - val_loss: 0.0687 - val_accuracy: 0.9787


In [34]:
model.save('ner.h5')

In [35]:
import pickle

In [36]:
pickle.dump(model, open('nermodel.pkl', 'wb'))

In [37]:
model = tf.keras.models.load_model("ner.h5")

In [38]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 110, 110)          3074940   
                                                                 
 dropout (Dropout)           (None, 110, 110)          0         
                                                                 
 bidirectional (Bidirectiona  (None, 110, 200)         168800    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 110, 200)         240800    
 nal)                                                            
                                                                 
 time_distributed (TimeDistr  (None, 110, 17)          3417      
 ibuted)                                                         
                                                        

In [39]:
model.evaluate(test_dataset)

75/75 [==============================] - 41s 511ms/step - loss: 0.0673 - accuracy: 0.9792


[0.06727100908756256, 0.979217529296875]